In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

## Data

## Loading data

In [3]:
PATH=Path('data/wikitext-2-raw')

In [4]:
bs = 64
bptt = 70

In [7]:
data = TextLMDataBunch.load(PATH, bs=bs, bptt=bptt)

### AWD-LSTM Model

In [8]:
learn = language_model_learner(data, clip=0.1)

In [9]:
rnn = learn.model

In [10]:
rnn

SequentialRNN(
  (0): RNNCore(
    (encoder): Embedding(29836, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(29836, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=29836, bias=True)
    (output_dp): RNNDropout()
  )
)

In [17]:
# encoder weights
rnn_enc = rnn[0].encoder; rnn_enc, rnn_enc.weight.shape

(Embedding(29836, 400, padding_idx=1), torch.Size([29836, 400]))

In [20]:
# decoder weights
rnn_dec = rnn[1].decoder; rnn_dec, rnn_dec.weight.shape

(Linear(in_features=400, out_features=29836, bias=True),
 torch.Size([29836, 400]))

Instead of using decoder, what if we use embeddings as a loss function instead